# Module 2: Deploy to Amazon Bedrock AgentCore

In this module, you'll deploy your Student Analytics Agent to **Amazon Bedrock AgentCore**.

## Learning Objectives

By the end of this module, you will:
- Configure your agent for AgentCore deployment
- Test locally using the AgentCore container
- Deploy to Amazon Bedrock AgentCore
- Invoke your deployed agent via HTTP API

## What is Amazon Bedrock AgentCore?

AgentCore provides:
- **Isolated microVM execution** - Secure sandbox for agent code
- **Built-in observability** - CloudWatch logging and tracing
- **Credential management** - Secure access to AWS services
- **Protocol support** - HTTP, MCP, Agent-to-Agent communication
- **Scalability** - Managed infrastructure

## Prerequisites

Before starting this module, ensure:
- Module 0 setup is complete
- Module 1a and 1b are completed
- Docker is installed and running

The `bedrock-agentcore-starter-toolkit` will be automatically installed if needed.

In [ ]:
import os
import sys
import subprocess
import json
from pathlib import Path
from dotenv import load_dotenv

# Set up paths
workshop_root = Path("..").resolve()
sys.path.insert(0, str(workshop_root))

# Load environment
load_dotenv(workshop_root / ".env")

# Ensure Bedrock is used
os.environ["CLAUDE_CODE_USE_BEDROCK"] = "1"

print(f"Workshop root: {workshop_root}")
print(f"AWS Region: {os.getenv('AWS_REGION')}")

In [ ]:
import boto3
from botocore.exceptions import ClientError, NoCredentialsError, CredentialRetrievalError

print("Checking prerequisites...")
print("=" * 60)

# Check AWS Credentials FIRST
print("\n1. AWS Credentials:")
try:
    sts = boto3.client('sts', region_name=os.getenv('AWS_REGION', 'us-west-2'))
    identity = sts.get_caller_identity()
    print(f"   ✅ Status: VALID")
    print(f"   Account: {identity['Account']}")
    print(f"   User/Role: {identity['Arn'].split('/')[-1]}")
except (ClientError, NoCredentialsError, CredentialRetrievalError) as e:
    print(f"   ❌ Status: EXPIRED or INVALID")
    print(f"   Error: {e}")
    print("\n   To refresh credentials, run ONE of these in a terminal:")
    print("   - aws sso login          # If using SSO")
    print("   - aws configure          # If using access keys")
    print("   - export AWS_PROFILE=... # If using named profiles")
    print("\n   Then restart this notebook kernel and re-run.")

# Check Docker
print("\n2. Docker:")
try:
    result = subprocess.run(["docker", "--version"], capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✅ {result.stdout.strip()}")
    else:
        print("   ❌ NOT FOUND - Please install Docker Desktop")
except FileNotFoundError:
    print("   ❌ NOT FOUND - Please install Docker Desktop")

# Check and install agentcore CLI if needed
print("\n3. AgentCore CLI:")
try:
    result = subprocess.run(["agentcore", "--help"], capture_output=True, text=True)
    if result.returncode == 0:
        print(f"   ✅ Installed (bedrock-agentcore-starter-toolkit)")
    else:
        raise FileNotFoundError("agentcore not working")
except FileNotFoundError:
    print("   ⏳ Not found - installing bedrock-agentcore-starter-toolkit...")
    
    # Try pip first (preferred), fall back to pip
    install_result = subprocess.run(
        ["pip", "install", "bedrock-agentcore-starter-toolkit"],
        capture_output=True, text=True
    )
    
    if install_result.returncode != 0:
        # Try pip if pip fails
        install_result = subprocess.run(
            [sys.executable, "-m", "pip", "install", "bedrock-agentcore-starter-toolkit"],
            capture_output=True, text=True
        )
    
    if install_result.returncode == 0:
        print("   ✅ Successfully installed bedrock-agentcore-starter-toolkit")
        print("   Note: You may need to restart your terminal/notebook kernel")
    else:
        print("   ❌ Installation failed:")
        print(f"   {install_result.stderr[:200]}")

print("\n" + "=" * 60)

## Step 1: Understand the AgentCore Agent

The AgentCore agent wraps our agent for deployment. Let's examine the structure.

In [ ]:
# Look at the AgentCore agent wrapper
agentcore_agent = workshop_root / "agent" / "agent_agentcore.py"

with open(agentcore_agent, 'r') as f:
    content = f.read()

print("AgentCore Agent Structure:")
print("=" * 50)
print(content[:2000])
print("\n... [truncated]")

## Step 1b: Generate Dockerfile

`agentcore configure` is commonly used to generate `Dockerfile` and `.bedrock_agentcore.yaml` which would prompt users for customisation options. For consistency in this workshop, we will craft these two files with our own code.

First, generate Dockerfile with your specific AWS configuration (region, S3 bucket, etc.).

In [ ]:
# Generate Dockerfile with environment-specific configuration
dockerfile_path = workshop_root / "Dockerfile"

# Get configuration from environment
aws_region = os.getenv('AWS_REGION', 'us-east-1')
athena_database = os.getenv('ATHENA_DATABASE', 'student_analytics')
athena_output = os.getenv('ATHENA_OUTPUT_LOCATION', '')
s3_bucket = os.getenv('S3_BUCKET_NAME', '')

# Construct ATHENA_OUTPUT_LOCATION if not set
if not athena_output and s3_bucket:
    athena_output = f"s3://{s3_bucket}/athena-results/"

print("Generating Dockerfile with your configuration...")
print("=" * 60)
print(f"AWS_REGION: {aws_region}")
print(f"ATHENA_DATABASE: {athena_database}")
print(f"ATHENA_OUTPUT_LOCATION: {athena_output}")
print("=" * 60)

dockerfile_content = f"""# Dockerfile for Amazon Bedrock AgentCore deployment
# Student Analytics AI Agent
# Generated by Module 2 notebook

FROM public.ecr.aws/docker/library/python:3.12-slim

WORKDIR /app

# Install system dependencies
RUN apt-get update && apt-get install -y --no-install-recommends \
    gcc \
    libpq-dev \
    && rm -rf /var/lib/apt/lists/*

# Copy requirements first for better caching
COPY requirements.txt .

# Install Python dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Copy the application code
COPY . .

# Create results directories
RUN mkdir -p /app/results/raw /app/results/processed

# Set environment variables
ENV PYTHONUNBUFFERED=1
ENV PYTHONDONTWRITEBYTECODE=1

# CRITICAL: Use AWS Bedrock as the model provider
ENV CLAUDE_CODE_USE_BEDROCK=1

# AWS and Athena configuration (from your .env)
ENV AWS_REGION={aws_region}
ENV ATHENA_DATABASE={athena_database}
ENV ATHENA_OUTPUT_LOCATION={athena_output}
ENV ANTHROPIC_MODEL=global.anthropic.claude-opus-4-5-20251101-v1:0
ENV ANTHROPIC_SMALL_FAST_MODEL=global.anthropic.claude-haiku-4-5-20251001-v1:0
ENV CLAUDE_CODE_MAX_OUTPUT_TOKENS=64000
ENV MAX_THINKING_TOKENS=4800
ENV DISABLE_PROMPT_CACHING=1
ENV CLAUDE_CODE_ENABLE_TELEMETRY=1

# Expose the port that AgentCore expects
EXPOSE 8080

# Basic deployment (no observability)
# Module 3b will update this to use opentelemetry-instrument
CMD ["python", "main.py"]
"""

with open(dockerfile_path, 'w') as f:
    f.write(dockerfile_content)

print("\n✅ Dockerfile generated successfully!")
print(f"   Location: {dockerfile_path}")

## Step 2: Configure AgentCore

The following code will generate `.bedrock_agentcore.yaml`, which is required to deploy to AgentCore. Usually this file can be generated by `agentcore configure`, which will prompt users for configuration options. For consistency in this workshop, we will use the following code to generate `.bedrock_agentcore.yaml`.

In [ ]:
import yaml
import platform

config_file = workshop_root / ".bedrock_agentcore.yaml"

# Get values from environment
aws_region = os.getenv('AWS_REGION', 'us-east-1')
aws_account_id = os.getenv('AWS_ACCOUNT_ID', '')

# Detect platform architecture
arch = 'linux/arm64' if platform.machine() == 'arm64' else 'linux/amd64'

# Define the execution role ARN
execution_role_arn = f"arn:aws:iam::{aws_account_id}:role/StudentAnalyticsAgentCoreRole"

if not config_file.exists():
    # Create a new AgentCore configuration
    config = {
        'default_agent': 'student_analytics_agent',
        'agents': {
            'student_analytics_agent': {
                'name': 'student_analytics_agent',
                'entrypoint': 'main.py',  # Module 2: basic deployment
                'deployment_type': 'container',
                'runtime_type': None,
                'platform': arch,
                'container_runtime': 'docker',
                'source_path': '.',
                'aws': {
                    'execution_role': execution_role_arn,
                    'execution_role_auto_create': False,
                    'account': aws_account_id,
                    'region': aws_region,
                    'ecr_auto_create': True,
                    's3_auto_create': False,
                    'network_configuration': {
                        'network_mode': 'PUBLIC',
                        'network_mode_config': None
                    },
                    'protocol_configuration': {
                        'server_protocol': 'HTTP'
                    },
                    'observability': {
                        'enabled': True
                    },
                    'lifecycle_configuration': {
                        'idle_runtime_session_timeout': None,
                        'max_lifetime': None
                    }
                },
                'bedrock_agentcore': {
                    'agent_id': None,
                    'agent_arn': None,
                    'agent_session_id': None
                },
                'codebuild': {
                    'project_name': None,
                    'execution_role': None,
                    'source_bucket': None
                },
                'memory': {
                    'mode': 'NO_MEMORY',
                    'memory_id': None,
                    'memory_arn': None,
                    'memory_name': None,
                    'event_expiry_days': 30,
                    'first_invoke_memory_check_done': False,
                    'was_created_by_toolkit': False
                },
                'identity': {
                    'credential_providers': [],
                    'workload': None
                },
                'aws_jwt': {
                    'enabled': False,
                    'audiences': [],
                    'signing_algorithm': 'ES384',
                    'issuer_url': None,
                    'duration_seconds': 300
                },
                'authorizer_configuration': None,
                'request_header_configuration': None,
                'oauth_configuration': None,
                'api_key_env_var_name': None,
                'api_key_credential_provider_name': None,
                'is_generated_by_agentcore_create': False
            }
        }
    }
    
    with open(config_file, 'w') as f:
        yaml.dump(config, f, default_flow_style=False, sort_keys=False, allow_unicode=True)
    
    print("✅ Created AgentCore configuration!")
    print(f"   Using execution role: {execution_role_arn}")
else:
    # Config exists - ensure entrypoint is main.py for Module 2
    with open(config_file, 'r') as f:
        config = yaml.safe_load(f)
    
    # Check and fix entrypoint if needed (might be main_observable.py from Module 3b)
    current_entrypoint = config['agents']['student_analytics_agent'].get('entrypoint', '')
    if current_entrypoint != 'main.py':
        config['agents']['student_analytics_agent']['entrypoint'] = 'main.py'
        with open(config_file, 'w') as f:
            yaml.dump(config, f, default_flow_style=False, sort_keys=False, allow_unicode=True)
        print(f"📝 Updated entrypoint from '{current_entrypoint}' to 'main.py' for Module 2")
    else:
        print("✅ Configuration already set for Module 2 (entrypoint: main.py)")

# Display current configuration
print("\n" + "=" * 60)
print("Current AgentCore Configuration:")
print("=" * 60)
with open(config_file, 'r') as f:
    config = yaml.safe_load(f)

agent_config = config['agents']['student_analytics_agent']
print(f"  Agent: student_analytics_agent")
print(f"  Entrypoint: {agent_config.get('entrypoint')}")
print(f"  Platform: {agent_config.get('platform')}")
print(f"  Region: {agent_config['aws'].get('region')}")
print(f"  Execution Role: {agent_config['aws'].get('execution_role')}")

deployed = agent_config.get('bedrock_agentcore', {})
if deployed.get('agent_id'):
    print(f"  Agent ID: {deployed.get('agent_id')}")
    print(f"  Status: Previously deployed")
else:
    print(f"  Status: Not yet deployed")

## Step 3: Local Testing with Dev Server (Optional)

Before deploying to AgentCore, you can host the agent code locally using `agentcore dev` and `agentcore invoke --dev '{"query": "How many students are currently enrolled?"}'`. This enable fast testing iteration. It supports hot reloading, automatically updating the local server when code changes occur, allowing rapid testing. For simplicity, we will do run local testing in the next two code cells in this notebook.

**Important Notes:**
- If you encounter HTTP 405 errors, kill any existing process on port 8080 and rerun the next cell

### Hosting the agent code locally simulating AgentCore Runtime environment.

In [ ]:
import subprocess
import os

print("Step 3: Local Testing with Dev Server (Optional)")
print("=" * 70)

dev_port = "8080"

def start_dev_server_background(port):
    """Start dev server in background with output to log file"""
    log_file = workshop_root / "dev_server.log"
    
    # Create the command to run in background
    # Use nohup and redirect output to log file to avoid blocking
    env = os.environ.copy()
    env["PYTHONUNBUFFERED"] = "1"
    
    # Start process with output redirected to file (not piped to notebook)
    with open(log_file, 'w') as f:
        process = subprocess.Popen(
            ["agentcore", "dev", "--port", str(port)],
            cwd=str(workshop_root),
            stdout=f,
            stderr=subprocess.STDOUT,
            env=env,
            start_new_session=True  # Detach from notebook process
        )
    
    return process.pid, log_file


pid, log_file = start_dev_server_background(dev_port)
print(f"   Process ID: {pid}")
print(f"   Log file: {log_file}")

### Run local test with sample query

In [ ]:
print("Invoking local agent...")
print("=" * 60)

query_text = "How many students are enrolled?"
payload = json.dumps({"query": query_text})

print(f"Query: {query_text}\n")

result = subprocess.Popen(
    ["agentcore", "invoke", "--dev", payload],
    cwd=str(workshop_root),
    stdout=subprocess.PIPE, 
    stderr=subprocess.STDOUT, 
    text=True, 
    bufsize=1
)

# Read output line by line as it becomes available
for line in result.stdout:
    print(line, end='', flush=True) # Print line and flush output to notebook

# Wait for the process to finish and get the return code
result.wait()

## Step 4: Deploy to AgentCore

Once local testing works, deploy to AgentCore on AWS.

This will:
1. Build a container image with your agent code
2. Push the image to ECR
3. Deploy to AgentCore service
4. Return the agent ARN and endpoint

In [ ]:
# Deploy to AgentCore (this takes 5-10 minutes)
print("Deploying to AgentCore...")
print("=" * 70)
print(f"\nWorking directory: {workshop_root}")
print(f"Config file exists: {(workshop_root / '.bedrock_agentcore.yaml').exists()}")
print("\nEstimated time: 5-10 minutes for first deployment")
print("=" * 70)

proceed = True

if proceed:
    print("\nStarting deployment...\n")
    
    # Run with full output capture
    result = subprocess.run(
        ["agentcore", "deploy"],
        cwd=str(workshop_root),
        capture_output=True,
        text=True,
        env={**os.environ, 'PYTHONUNBUFFERED': '1'}
    )
    
    # Show stdout
    if result.stdout:
        print("STDOUT:")
        print("-" * 70)
        print(result.stdout)
    
    # Show stderr
    if result.stderr:
        print("\nSTDERR:")
        print("-" * 70)
        print(result.stderr)
    
    print("\n" + "=" * 70)
    if result.returncode == 0:
        print("Deployment completed successfully!")
    else:
        print(f"Deployment failed (exit code: {result.returncode})")
        print("\nTroubleshooting:")
        print("  1. Check AWS credentials: aws sts get-caller-identity")
        print("  2. Check config file: cat .bedrock_agentcore.yaml")
        print("  3. Try running manually: cd .. && agentcore deploy")
    print("=" * 70)
else:
    print("\nDeployment skipped. Run this cell again when ready.")

### Check deployment status

In [ ]:
# Check deployment status
print("Checking AgentCore status...")
print("=" * 60)

result = subprocess.run(
    ["agentcore", "status"],
    cwd=str(workshop_root),
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print(result.stdout)
else:
    if "not yet deployed" in result.stderr.lower() or "not deployed" in result.stdout.lower():
        print("Agent not yet deployed. Run the deployment cell above.")
    else:
        print("Error checking status:")
        print(result.stderr or result.stdout)

## Step 5: Invoke the Deployed Agent

Once deployed, invoke your agent using the AgentCore API.

In [ ]:
print("Invoking local agent...")
print("=" * 60)

query_text = "How many students are enrolled?"
payload = json.dumps({"query": query_text})

print(f"Query: {query_text}\n")

result = subprocess.Popen(
    ["agentcore", "invoke", payload],
    cwd=str(workshop_root),
    stdout=subprocess.PIPE, 
    stderr=subprocess.STDOUT, 
    text=True, 
    bufsize=1
)

# Read output line by line as it becomes available
for line in result.stdout:
    print(line, end='', flush=True) # Print line and flush output to notebook

# Wait for the process to finish and get the return code
result.wait()

## Step 6: Re-run Module 1 Queries on Deployed Agent

Let's verify the deployed agent works just like our local agent.

In [ ]:
import subprocess
import json
import re

# Test queries from Module 1
test_queries = [
    "How many students are currently enrolled?",
    "What is the average scholarship coverage rate for students in each major?"
]

print("Running test queries on deployed agent...")
print("=" * 70)

# Function to strip ANSI color codes for cleaner output
def strip_ansi(text):
    ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
    return ansi_escape.sub('', text)

for i, query_text in enumerate(test_queries, 1):
    print(f"\n{'='*70}")
    print(f"Query {i}: {query_text}")
    print("=" * 70)
    
    payload = json.dumps({"query": query_text})
    
    try:
        result = subprocess.Popen(
            ["agentcore", "invoke", payload],
            cwd=str(workshop_root),
            stdout=subprocess.PIPE, 
            stderr=subprocess.STDOUT, 
            text=True, 
            bufsize=1
        )

        # Read output line by line as it becomes available
        for line in result.stdout:
            print(line, end='', flush=True) # Print line and flush output to notebook

        # Wait for the process to finish and get the return code
        result.wait()
        
    except Exception as e:
        print(f"\n❌ Error: {type(e).__name__}: {e}")
    
    print()

print("\n" + "=" * 70)
print("✅ All test queries completed!")
print("=" * 70)

## Key Takeaways

### What We Accomplished

1. **Configured** AgentCore deployment settings
2. **Tested locally** with dev server before deployment
3. **Deployed** to production AgentCore
4. **Invoked** the deployed agent via CLI and API

### AgentCore Benefits

- **Security** - Isolated microVM execution
- **Scalability** - Managed infrastructure
- **Observability** - Built-in logging and tracing
- **Simplicity** - Deploy with single command

## Next Steps

Continue to:

**[Module 3: Observability](../module-3-observability/3-agentcore-observability.ipynb)** (Stretch Goal)

In the next module, you'll:
- Enable CloudWatch logging
- Trace agent execution
- Monitor performance metrics

---

*Workshop: Build Agentic AI Applications with Claude Agent SDK and Amazon Bedrock AgentCore*